In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import keras_tuner as kt

from numpy import array
from numpy import hstack
from numpy import empty
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from os import chdir
from os import getcwd
from os import listdir
import math

csv_file_path = 'D:/Arbeit PhD/Fachlich/z_Sonstiges/Groundwater challenge/data/USA'
chdir(csv_file_path)

Y_all = pd.read_csv('heads.csv',decimal='.',index_col=0, delimiter=',', header=0,parse_dates=True)
X_all = pd.read_csv('input_data.csv',decimal='.', delimiter=',',index_col=0, header=0,parse_dates=True)
Y_temp = Y_all['2002-03-01':' 2016-12-26']
X = X_all['2002-03-01':' 2016-12-26']
print()

# interpolate
Y = Y_temp.reindex(pd.date_range(start=Y_temp.index.min(),end=Y_temp.index.max(),freq='1D')) 
Y.interpolate(method='linear', inplace=True)  
print(X.shape, Y.shape)
print(Y.isna().sum().sum())
X


(5415, 5) (5415, 1)
0


,PRCP,TMAX,TMIN,Stage_m,ET
2002-03-01,0.000,6.666667,-11.111111,0.923499,1.380044
2002-03-02,0.000,7.222222,-8.333333,0.911308,1.447215
2002-03-03,28.956,15.555556,2.222222,1.161231,2.115481
2002-03-04,0.254,2.222222,-7.777778,1.203901,1.031090
2002-03-05,0.000,0.000000,-11.111111,1.085035,0.893230
...,...,...,...,...,...
2016-12-22,0.000,2.222222,-11.111111,0.819872,0.563036
2016-12-23,0.000,5.555556,-3.888889,0.819872,0.665206
2016-12-24,0.000,5.555556,-1.111111,0.862542,0.602258
2016-12-25,9.906,5.555556,0.555556,0.981408,0.544738


In [2]:
X_train = X[:'2011-12-31']
X_test = X['2014-01-01':]
X_valid = X['2012-01-01':'2013-12-31']

Y_train = Y[:'2011-12-31']
Y_test = Y['2014-01-01':]
Y_valid = Y['2012-01-01':'2013-12-31']
print(X_test.shape)
print(Y_test.shape)

csv_file_path = 'D:\Arbeit PhD\Fachlich\z_Sonstiges\Groundwater challenge\data'
chdir(csv_file_path)
from helper import *

(dataset_train,dataset_test,dataset_valid,scaler_X,scaler_Y)=prepare_data(X_train,Y_train,X_valid,Y_valid,X_test,Y_test)

x_final_unsplit = scaler_X.transform(X)
y_final_unsplit = scaler_Y.transform(Y)
dataset_train = np.concatenate((x_final_unsplit,y_final_unsplit), axis=1)
 
iters=3
n_steps_in=30
n_steps_out=1

x_final, y_final = split_sequences_y1(dataset_train, n_steps_in, n_steps_out)

(1091, 5)
(1091, 1)


In [3]:

import keras_tuner as kt
from tensorflow.keras.layers import Dropout
from tensorflow.keras import backend as K
class Dropout(keras.layers.Dropout):
    def __init__(self, rate, training=None, noise_shape=None, seed=None, **kwargs):
        super(Dropout, self).__init__(rate, noise_shape=None, seed=None,**kwargs)
        self.training = training

        
    def call(self, inputs, training=None):
        if 0. < self.rate < 1.:
            noise_shape = self._get_noise_shape(inputs)

            def dropped_inputs():
                return K.dropout(inputs, self.rate, noise_shape,
                                 seed=self.seed)
            if not training: 
                return K.in_train_phase(dropped_inputs, inputs, training=self.training)
            return K.in_train_phase(dropped_inputs, inputs, training=training)
        return inputs

In [4]:
test_runs = 100

class MonteCarloLSTM(tf.keras.layers.LSTM):
    def call(self, inputs):
        return super().call(inputs, training=True)
    
#load models 
csv_file_path = 'D:\Arbeit PhD\Fachlich\z_Sonstiges\Groundwater challenge'
chdir(csv_file_path)

transformer_model = keras.models.load_model('TF_USA_tune.h5')
transformer_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 30, 5)]      0                                            
__________________________________________________________________________________________________
layer_normalization (LayerNorma (None, 30, 5)        10          input_1[0][0]                    
__________________________________________________________________________________________________
multi_head_attention (MultiHead (None, 30, 5)        1454        layer_normalization[0][0]        
                                                                 layer_normalization[0][0]        
__________________________________________________________________________________________________
dropout (Dropout)               (None, 30, 5)        0           multi_head_attention[0][0]   

## Prediction Transformer

In [5]:

print(x_final.shape)
Y_pred_test_tr = np.empty((x_final.shape[0],n_steps_out,test_runs))
y_correct_utf = scaler_Y.inverse_transform(y_final)#test
for j in range(test_runs): 
    Y_pred_test_tr[:,:,j]=transformer_model.predict(x_final)
    yhat_utf = scaler_Y.inverse_transform(Y_pred_test_tr[:,:,j])#test
    #print(j)

(5386, 30, 5)


## POC

In [6]:
Y_pred = np.mean(Y_pred_test_tr, axis=2)
Y_std = np.std(Y_pred_test_tr, axis=2)

zT = 5
T_ival = [Y_pred - zT*Y_std,Y_pred + zT*Y_std]
#print(range(len(Y_test_day1)))
def calc_POC(list_of_bounds, measured):
    temp = 0 
    for i in range(len(measured)):
        if measured[i]< list_of_bounds[1][i] and measured[i]> list_of_bounds[0][i]:
            temp = temp + 1
    POC = temp/len(measured)
    return POC

print(T_ival[1].shape)
POC_tr = calc_POC(T_ival,y_final)
print(POC_tr)

(5386, 1)
0.9961010025993317


In [7]:
print(Y_pred_test_tr.shape)
Y_pred_test_tr_flat = Y_pred_test_tr.reshape((test_runs*Y_pred_test_tr.shape[0], 1))
Y_pred_test_utf_flat = scaler_Y.inverse_transform(Y_pred_test_tr_flat)
Y_pred_test_utf = Y_pred_test_utf_flat.reshape((Y_pred_test_tr.shape[0], 1,test_runs))
Y_pred_calib = np.mean(Y_pred_test_utf, axis=2)
Y_std_calib = np.std(Y_pred_test_utf, axis=2)
T_ival_calib = [Y_pred_calib - zT*Y_std_calib,Y_pred_calib + zT*Y_std_calib]
T_ival_calib[1]

(5386, 1, 100)


array([[153.84882399],
       [153.70788301],
       [153.73007007],
       ...,
       [151.35421565],
       [151.44139385],
       [151.63149344]])

In [8]:
csv_file_path = 'D:/Arbeit PhD/Fachlich/z_Sonstiges/Groundwater challenge/data/USA'
chdir(csv_file_path)

X_all = pd.read_csv('input_data.csv',decimal='.',index_col=0, delimiter=',', header=0,parse_dates=True)

X_finaltest = X_all['2017-01-01':]
X_finaltest

,PRCP,TMAX,TMIN,Stage_m,ET
2017-01-01,1.778,4.444444,-9.444444,0.950930,0.675063
2017-01-02,0.000,5.000000,-7.222222,0.923499,0.694637
2017-01-03,1.270,1.666667,-6.666667,0.947882,0.527591
2017-01-04,17.526,3.333333,1.111111,1.161231,0.359905
2017-01-05,0.000,7.777778,-5.555556,1.103322,0.836176
...,...,...,...,...,...
2022-05-27,0.000,21.111111,8.888889,0.914355,4.372622
2022-05-28,17.526,25.555556,15.555556,0.978360,4.659461
2022-05-29,12.192,22.222222,8.888889,1.222188,4.665706
2022-05-30,0.000,25.000000,9.444444,1.091131,5.309803


In [9]:
X_finaltest_trf = scaler_X.transform(X_finaltest)
X_ftest = split_sequences_test(X_finaltest_trf,n_steps_in, n_steps_out)
X_ftest.shape

(1948, 30, 5)

In [10]:
print(x_final.shape)
Y_pred_ftest_tr = np.empty((X_ftest.shape[0],n_steps_out,test_runs))
for j in range(test_runs): 
    Y_pred_ftest_tr[:,:,j]=transformer_model.predict(X_ftest)

(5386, 30, 5)


In [11]:
Y_pred_ftest_tr_flat = Y_pred_ftest_tr.reshape((100*Y_pred_ftest_tr.shape[0], 1))
Y_pred_ftest_utf_flat = scaler_Y.inverse_transform(Y_pred_ftest_tr_flat)
Y_pred_ftest_utf = Y_pred_ftest_utf_flat.reshape((Y_pred_ftest_tr.shape[0], 1,100))
yhat_mean = np.mean(Y_pred_ftest_utf, axis=2)
yhat_std = np.std(Y_pred_ftest_utf, axis=2)
T_ival_utf = [yhat_mean - zT*yhat_std,yhat_mean + zT*yhat_std]
yhat_mean.shape

(1948, 1)

In [12]:
final_idx_calib = pd.date_range(start='2002-03-30', end='2016-12-26')# 2002-03-01 to 2016-12-31
final_idx_test = pd.date_range(start='2017-01-30', end='2022-05-31')#2017-01-01 to 2022-05-31
final_df_calib = pd.DataFrame(index=final_idx_calib)
final_df_test = pd.DataFrame(index=final_idx_test)
final_df_test.shape

(1948, 0)

In [13]:
print(Y_pred.shape)
final_df_calib['head forecast'] = Y_pred_calib
final_df_calib['95% lower'] = T_ival_calib[0]
final_df_calib['95% upper'] = T_ival_calib[1]
final_df_calib

(5386, 1)


,head forecast,95% lower,95% upper
2002-03-30,152.599119,151.349413,153.848824
2002-03-31,152.586810,151.465738,153.707883
2002-04-01,152.661618,151.593165,153.730070
2002-04-02,152.659798,151.417166,153.902430
2002-04-03,152.750312,151.566397,153.934228
...,...,...,...
2016-12-22,149.943222,148.648636,151.237809
2016-12-23,149.957712,148.567568,151.347855
2016-12-24,150.004606,148.654996,151.354216
2016-12-25,149.975343,148.509292,151.441394


In [14]:
final_df_test['head forecast'] = yhat_mean
final_df_test['95% lower'] = T_ival_utf[0]
final_df_test['95% upper'] = T_ival_utf[1]
final_df_test

,head forecast,95% lower,95% upper
2017-01-30,152.168012,151.121386,153.214638
2017-01-31,152.104987,151.001279,153.208695
2017-02-01,152.029914,151.135706,152.924122
2017-02-02,152.091838,151.128365,153.055312
2017-02-03,152.161370,151.191194,153.131546
...,...,...,...
2022-05-27,152.351945,151.433442,153.270448
2022-05-28,152.337127,151.505844,153.168410
2022-05-29,152.281888,151.336601,153.227175
2022-05-30,152.268237,151.352588,153.183885


In [15]:
csv_file_path = 'D:/Arbeit PhD/Fachlich/z_Sonstiges/Groundwater challenge/submissions'
chdir(csv_file_path)
final_df_calib.to_csv('USA_calib.csv', float_format='%.2f')
final_df_test.to_csv('USA_test.csv', float_format='%.2f')